#**Convexity Trading and Bond Portfolio Risk Management**

##**Function for discount factors from spot rates**

In [1]:
def disc_fact(spot):
  discount_factors=[]
  for i in range(len(spot)):
    discount_factors.append(1/(1+spot[i])**(i+1))
  return discount_factors

In [2]:
#Execute the above function with an example
spot_rates = [0.007, 0.015, 0.023, 0.030, 0.037, 0.043, 0.048, 0.052, 0.056, 0.06]
spot_rates

disc_fact(spot_rates)

[0.99304865938431,
 0.9706617486471405,
 0.934056396477656,
 0.8884870479156888,
 0.8338851080567814,
 0.7767730500853498,
 0.7202296919196319,
 0.666613464512311,
 0.612385270513089,
 0.5583947769151179]

##**Function for bond price from spot rates**

In [3]:
def bond_price_spot(cpn, spot, tenor, principal):
  price = 0
  z = disc_fact(spot)
  for i in range(tenor):
    if(i == tenor-1):
      price = price +  principal*(1+cpn)*z[i]
    else:
      price = price +  principal*cpn*z[i]
  return price

In [4]:
bond_price_spot(0.1,spot_rates,5,100)

129.58990041049393

##**Function for bond prices from YTM**

In [5]:
def bond_price_ytm(cpn, ytm, tenor, principal):
  return ((1-(1+ytm)**(-1*tenor))*(cpn*principal/ytm)) + (principal*((1+ytm)**(-1*tenor)))

In [6]:
bond_price_ytm(0.05, 0.05, 5, 100)

100.0

##**YTM Calculation using Newton Raphson**

### 1. Finite difference for first derivative calculation for Newton Raphson

In [7]:
def func_val(cpn, ytm, tenor, principal, fair_price):
  return bond_price_ytm(cpn, ytm, tenor, principal) - fair_price

def func_deriv(cpn, ytm, tenor, principal, fair_price, shock_size):
  up_shock_output = func_val(cpn, ytm+shock_size, tenor, principal, fair_price)
  down_shock_output = func_val(cpn, ytm-shock_size, tenor, principal, fair_price)
  return ((up_shock_output - down_shock_output)/(2*shock_size))

###Python function for Newton Raphson for YTM calculation

In [8]:
def newtonRaphson_ytm(guess_ytm, max_iter, tol_val, cpn, tenor, principal, shock_size, fair_price):
  nb_iter = 0
  diff = func_val(cpn, guess_ytm, tenor, principal, fair_price)/func_deriv(cpn, guess_ytm, tenor, principal, fair_price, shock_size)
  if(abs(diff)>tol_val):
    for i in range(max_iter):
      if(abs(diff)>tol_val):
        guess_ytm = guess_ytm - diff
        diff = func_val(cpn, guess_ytm, tenor, principal, fair_price)/func_deriv(cpn, guess_ytm, tenor, principal, fair_price, shock_size)
        nb_iter+=1 #nb_iter = nb_iter +1
      else:
        exit
    #else:
      #print("The value of the root is : ", "%.5f"% guess_ytm)
      #return 
    
    #print("The value of the root is : ", "%.5f"% guess_ytm)
  return guess_ytm 

In [9]:
newtonRaphson_ytm(0.1165, 1000, 0.000001, 0.09, 8, 100, 0.001, 111.94259701242748)

0.06999965735529554

##**Functions for modified duration and modified convexity**

### 1. Modified Duration

In [10]:
def bond_modified_duration(cpn, ytm, tenor, principal, shock_size):
  base_price = bond_price_ytm(cpn, ytm, tenor, principal)
  up_shock_output = bond_price_ytm(cpn, ytm+shock_size, tenor, principal)
  down_shock_output = bond_price_ytm(cpn, ytm-shock_size, tenor, principal)
  return ((down_shock_output-up_shock_output)/(2*base_price*shock_size))

In [11]:
bond_modified_duration(0.09, 0.15, 5, 100, 0.001)

3.600034447830464

### 2. Modified Convexity 

In [12]:
def bond_modified_convexity(cpn, ytm, tenor, principal, shock_size):
  up_shock_output = bond_price_ytm(cpn, ytm+shock_size, tenor, principal)
  down_shock_output = bond_price_ytm(cpn, ytm-shock_size, tenor, principal)
  base_price = bond_price_ytm(cpn, ytm, tenor, principal)
  return ((down_shock_output+up_shock_output-2*base_price)/(base_price*shock_size*shock_size))

In [13]:
bond_modified_convexity(0.09, 0.07, 5, 100, 0.001)

21.15735693606324

##**Portfolio Construction for convexity trade**

### 1. Calculation of bond price and bond YTM for all the bonds in the portfolio

In [14]:
cpn_5Y = float(input("Enter Coupon for 5Y bond: "))
cpn_2Y = float(input("Enter Coupon for 2Y bond: "))
cpn_8Y = float(input("Enter Coupon for 8Y bond: "))

#Note, the total principal for the bond = number of bonds * principal of 1 bond
prin_5Y = float(input("Enter Total Princpal for 5Y bond: "))
prin_2Y = float(input("Enter Total Princpal for 2Y bond: "))
prin_8Y = float(input("Enter Total Princpal for 8Y bond: "))

#cpn_5Y = 0.1
#cpn_2Y = 0.07
#cpn_8Y = 0.14

#prin_5Y = 100
#prin_2Y = 100
#prin_8Y = 100

bond_price_5Y = bond_price_spot(cpn_5Y,spot_rates,5,prin_5Y)
bond_price_2Y = bond_price_spot(cpn_2Y,spot_rates,2,prin_2Y)
bond_price_8Y = bond_price_spot(cpn_8Y,spot_rates,8,prin_8Y)

Enter Coupon for 5Y bond: 0.1
Enter Coupon for 2Y bond: 0.07
Enter Coupon for 8Y bond: 0.14
Enter Total Princpal for 5Y bond: 100
Enter Total Princpal for 2Y bond: 100
Enter Total Princpal for 8Y bond: 100


In [15]:
print(bond_price_5Y)
print(bond_price_2Y)
print(bond_price_8Y)

129.58990041049393
110.8121477209342
161.63391878921527


In [16]:
bond_ytm_5Y = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_5Y, 5, prin_5Y, 0.001, 129.58990041049393)
bond_ytm_2Y = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_2Y, 2, prin_2Y, 0.001, 110.8121477209342)
bond_ytm_8Y = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_8Y, 8, prin_8Y, 0.001, 161.63391878921527)

In [17]:
print(bond_ytm_5Y)
print(bond_ytm_2Y)
print(bond_ytm_8Y)

0.034547791599101124
0.014740982517686735
0.04612642118374499


### 2.Calculation of dollar duration and dollar convexity of all the bonds in the portfolio

Modified Duration = %change in portfolio value due 1% change in yield corresponding to first order sensitivity
Duration = -(1/P)*(dP/dY)

Dollar Duration = Dollar change in the value of portfolio for 1% change in yield based on first order sensitivity
Dollar Duration = Duration * Portfolio Value

Modified Convexity = %change in portfolio value due 1% change in yield corresponding to second order sensitivity
Duration = (1/P)*(d2P/dY2)

Dollar Convexity = Dollar change in the value of portfolio for 1% change in yield  based on second order sensitivity
Dollar Duration = Convexity * Portfolio Value

In [18]:
dollar_duration_5Y = bond_modified_duration(cpn_5Y, bond_ytm_5Y, 5, prin_5Y, 0.001)*bond_price_5Y
dollar_duration_2Y = bond_modified_duration(cpn_2Y, bond_ytm_2Y, 2, prin_2Y, 0.001)*bond_price_2Y
dollar_duration_8Y = bond_modified_duration(cpn_8Y, bond_ytm_8Y, 8, prin_8Y, 0.001)*bond_price_8Y

In [19]:
print(dollar_duration_5Y)
print(dollar_duration_2Y)
print(dollar_duration_8Y)

535.9506832881415
211.60709719788346
907.7798910829376


In [20]:
dollar_convexity_5Y = bond_modified_convexity(cpn_5Y, bond_ytm_5Y, 5, prin_5Y, 0.001)*bond_price_5Y
dollar_convexity_2Y = bond_modified_convexity(cpn_2Y, bond_ytm_2Y, 2, prin_2Y, 0.001)*bond_price_2Y
dollar_convexity_8Y = bond_modified_convexity(cpn_8Y, bond_ytm_8Y, 8, prin_8Y, 0.001)*bond_price_8Y

In [21]:
print(dollar_convexity_5Y)
print(dollar_convexity_2Y)
print(dollar_convexity_8Y)

2936.5547129816155
618.8997934422958
6909.778941967457


### 3. Determination of quantities required for replicating duration neutral and cost neutral portfolio

Assume n1 and n2 are the units to trade, then, for the replicating portfolio:

n1*P_2Y + n2*P_8Y = P_5Y ------------(1)

n1*DD_2Y + n2*DD_8Y = DD_5Y -----------(2)

Matrix Multiplication can be used to solve these:



In [22]:
import numpy as np

In [48]:
matrix_1 = np.array([[bond_price_2Y, bond_price_8Y],[dollar_duration_2Y, dollar_duration_8Y]])
print(matrix_1)
print(type(matrix_1))

[[110.81214772 161.63391879]
 [211.6070972  907.77989108]]
<class 'numpy.ndarray'>


In [49]:
matrix_2 = np.array([bond_price_5Y, dollar_duration_5Y])
print(matrix_2)
print(type(matrix_2))

[129.58990041 535.95068329]
<class 'numpy.ndarray'>


In [25]:
matrix_3 = np.linalg.inv(matrix_1)
print(matrix_3)
type(matrix_3)

[[ 0.01367341 -0.00243461]
 [-0.00318733  0.00166911]]


numpy.ndarray

In [26]:
nb_units = np.matmul(matrix_3,matrix_2)

In [27]:
n1 = nb_units[0]
n2 = nb_units[1]

print("The quantity of 2Y bond for replicating portfolio should be: ","%.10f"% n1)
print("The quantity of 8Y bond for replicating portfolio should be: ","%.10f"% n2)

The quantity of 2Y bond for replicating portfolio should be:  0.4671068542
The quantity of 8Y bond for replicating portfolio should be:  0.4815127126


##**PnL Analysis for the Portfolio based on Partial Reval (using TAYLOR'S APPROXIMATION)**




###1. Duration based PnL

**Duration based PnL (Taylors Approximation first order term)
= - nb_of_units * Dollar Duration * change in yield**

In [59]:
#Note: Duration_PnL_5Y is roughly equal to Duration_PnL_Portfolio with only parallel shift i.e. same change in yield across all tenors
# If the change in yields are applied across 2Y, 5Y, and 8Y, then Duration PnL don't offset with each other
chg_yield_2Y = float(input("Enter yield change for 5Y tenor: "))
chg_yield_5Y = float(input("Enter yield change for 2Y tenor: "))
chg_yield_8Y = float(input("Enter yield change for 8Y tenor: "))

#Short 1 unit of 5Y bond, and long n1 and n2 units of 2Y and 8Y bonds respective 
#Duration_PnL = - nb_units * Duration * Price * chg_yield = - nb_units * Dollar Duration * chg_yield
Duration_PnL_5Y = (- (-1) * dollar_duration_5Y * chg_yield_5Y)
Duration_PnL_2Y_8Y = (- (n1) * dollar_duration_2Y * chg_yield_2Y) + (- (n2) * dollar_duration_8Y * chg_yield_8Y)
Duration_PnL_Portfolio = Duration_PnL_5Y + Duration_PnL_2Y_8Y

print("Duration PnL for short leg of the portfolio is: ", "%.2f"% Duration_PnL_5Y)
print("Duration PnL for long leg of the portfolio is: ", "%.2f"% Duration_PnL_2Y_8Y)
print("Duration PnL for the total portfolio is: ", "%.2f"% Duration_PnL_Portfolio)

Enter yield change for 5Y tenor: -0.1
Enter yield change for 2Y tenor: -0.1
Enter yield change for 8Y tenor: -0.1
Duration PnL for short leg of the portfolio is:  -53.60
Duration PnL for long leg of the portfolio is:  53.60
Duration PnL for the total portfolio is:  0.00


Duration PnL for the total portfolio = 0 implies fully hedged portfolio based on duration measure. 

**Note: Duration PnL will not be 0 if the yield are shocked in non-parallel fashion**

###2. Convexity based PnL

**Convexity based PnL (Taylors Approximation second order term)
= 0.5* nb_of_units * Dollar Convexity * (change in yield)^2** 

In [60]:
#Based on Taylor's Approximation: Convexity_PnL = (0.5) * nb_units * Convexity * Price * chg_yield * chg_yield = 0.5* nb_units * Dollar Duration * chg_yield * chg_yield
#Convexity PnLs are linearly additive in a portfolio

Convexity_PnL_5Y = (0.5* (-1) * dollar_convexity_5Y * chg_yield_5Y * chg_yield_5Y)
Convexity_PnL_2Y_8Y = (0.5* (n1) * dollar_convexity_2Y * chg_yield_2Y * chg_yield_2Y) + (0.5 * (n2) * dollar_convexity_8Y * chg_yield_8Y * chg_yield_8Y)
Convexity_PnL_Portfolio = Convexity_PnL_5Y + Convexity_PnL_2Y_8Y

print("Convexity PnL for short leg of the portfolio is: ", "%.2f"% Convexity_PnL_5Y)
print("Convexity PnL for long leg of the portfolio is: ", "%.2f"% Convexity_PnL_2Y_8Y)
print("Convexity PnL for the total portfolio is: ", "%.2f"% Convexity_PnL_Portfolio)

Convexity PnL for short leg of the portfolio is:  -14.68
Convexity PnL for long leg of the portfolio is:  18.08
Convexity PnL for the total portfolio is:  3.40


###3. Total PnL (Duration and Convexity based)

In [61]:
Total_PnL_5Y = Duration_PnL_5Y + Convexity_PnL_5Y
Total_PnL_2Y_8Y = Duration_PnL_2Y_8Y + Convexity_PnL_2Y_8Y
Total_PnL_Portfolio = Total_PnL_5Y + Total_PnL_2Y_8Y

print("Total PnL for short leg of the portfolio is: ", "%.2f"% Total_PnL_5Y)
print("Total PnL for long leg of the portfolio is: ", "%.2f"% Total_PnL_2Y_8Y)
print("Total PnL for the entire portfolio is: ", "%.2f"% Total_PnL_Portfolio)

Total PnL for short leg of the portfolio is:  -68.28
Total PnL for long leg of the portfolio is:  71.68
Total PnL for the entire portfolio is:  3.40


###**Exercise**: Partial Reval PnL for variety of shocks (Scenario Analysis):
 
Perform the PnL Scenario analysis on change in yields with positive shock scenarios of [0.001, 0.002, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05] and with negative shock scenarios of [- 0.001, - 0.002, - 0.005, - 0.01, - 0.02, - 0.03, - 0.04, - 0.05]. 

Store the results in two different lists


##**PnL Analysis for the portfolio based on Full Revaluation**

###1. Change in prices of individual bonds

In [62]:
new_price_5Y = bond_price_ytm(cpn_5Y, bond_ytm_5Y + chg_yield_5Y, 5, prin_5Y)
new_price_2Y = bond_price_ytm(cpn_2Y, bond_ytm_2Y + chg_yield_2Y, 2, prin_2Y)
new_price_8Y = bond_price_ytm(cpn_8Y, bond_ytm_8Y + chg_yield_8Y, 8, prin_8Y)

chg_price_5Y = new_price_5Y - bond_price_5Y
chg_price_2Y = new_price_2Y - bond_price_2Y
chg_price_8Y = new_price_8Y - bond_price_8Y

print("New prices")
print(new_price_5Y)
print(new_price_2Y)
print(new_price_8Y)

print("\nChange in prices")
print(chg_price_5Y)
print(chg_price_2Y)
print(chg_price_8Y)

New prices
201.81836317982646
135.52798628875394
300.5985676780385

Change in prices
72.22846276933254
24.71583856781973
138.9646488888232


###2. Change in portfolio value

In [63]:
original_value_short_leg = 1*bond_price_5Y
original_value_long_leg = n1*bond_price_2Y + n2*bond_price_8Y
original_value_Portfolio = original_value_long_leg - original_value_short_leg

new_value_short_leg = 1* new_price_5Y
new_value_long_leg = n1*new_price_2Y + n2*new_price_8Y
new_value_Portfolio = new_value_long_leg - new_value_short_leg

chg_value_short_leg = new_value_short_leg - original_value_short_leg
chg_value_long_leg = new_value_long_leg - original_value_long_leg
chg_value_Portfolio = new_value_Portfolio - original_value_Portfolio

#Note that in first print function, it is -1*chg_value_short_leg as for the short leg, a positive change means loss and negative change means profit
print("Original Leg and Portfolio Values:")
print("Original_Portfolio_short_leg: ", original_value_short_leg)
print("Original_Portfolio_long_leg: ", original_value_long_leg)
print("Initial Cost to fund the portfolio: ","%.4f"% original_value_Portfolio)

print("\nLeg and Portfolio PnL:")
print("Total PnL for short leg of the portfolio is: ", "%.2f"% -chg_value_short_leg)
print("Total PnL for long leg of the portfolio is: ", "%.2f"% chg_value_long_leg)
print("Total PnL for the entire portfolio is: ", "%.2f"% chg_value_Portfolio)

Original Leg and Portfolio Values:
Original_Portfolio_short_leg:  129.58990041049393
Original_Portfolio_long_leg:  129.58990041049398
Initial Cost to fund the portfolio:  0.0000

Leg and Portfolio PnL:
Total PnL for short leg of the portfolio is:  -72.23
Total PnL for long leg of the portfolio is:  78.46
Total PnL for the entire portfolio is:  6.23


####Note: If the units are based on matrix equation used above, then its a price and duration replication strategy, which results in same original price and original duration of long and short leg. Thus, its a duration neutral zero cost portfolio. 

This is speculation example, as the trader is betting that his expectations on the volatility (and thus the yield shocks) will be realized and the profit here will be more than the reduction in portfolio value with passage of time and other changes. Also, this strategy will work only with parallel shifts in the yield at tenor points.

##**Generation of Spot discount factors at t = 0, and forward rates, single period forward discount factors, spot discount factors, and spot rate curves at t = 0, t = 1, and t = 2**

Note: The long route here has been taken to demonstrate the logical connection between different rates and discount factors today and at different points in time in the future



###**1.** Calculation of Spot discount factors at t=0 based on spot rates 

In [33]:
spot_rate_0Y = spot_rates
nb_of_term_points = len(spot_rates)

disc_fact_0Y_list = disc_fact(spot_rate_0Y)
disc_fact_0Y_list_orig = list.copy(disc_fact_0Y_list)
disc_fact_0Y_list_orig

[0.99304865938431,
 0.9706617486471405,
 0.934056396477656,
 0.8884870479156888,
 0.8338851080567814,
 0.7767730500853498,
 0.7202296919196319,
 0.666613464512311,
 0.612385270513089,
 0.5583947769151179]

1. Doing 1/disc_fact_0Y_list to generate compounding factors a.k.a (1+spot)^n will give error as list don't support that function. This can be dealt by converting it to array and then applying inverse. 

2. Alternatively, it can be done using lists which uses for loop (condensed in a list comprehension) - See https://www.geeksforgeeks.org/python-dividing-two-lists/

3. **For details on python lists**, refer https://www.programiz.com/python-programming/list

###**2.** Calculation of single period forward rates at t = 0 based on spot discount factors

In [34]:
#Inserting a '1' in the beginning and dropping the last element. 
#Note that running this multiple times will keep on inserting everytime. For that reason, this is being initialized to original list (disc_fact_0Y_list_orig) in the first line
#Note that there are more efficient ways to do it but it is explained in this way for simplicity of explanation in multiple steps
disc_fact_0Y_list_shift = list.copy(disc_fact_0Y_list_orig)
disc_fact_0Y_list_shift.insert(0,1)
print("Added 1 as the first element in the list: ", disc_fact_0Y_list_shift)
disc_fact_0Y_list_shift.pop(nb_of_term_points)
print("Removed last element from the list: ", disc_fact_0Y_list_shift)

Added 1 as the first element in the list:  [1, 0.99304865938431, 0.9706617486471405, 0.934056396477656, 0.8884870479156888, 0.8338851080567814, 0.7767730500853498, 0.7202296919196319, 0.666613464512311, 0.612385270513089, 0.5583947769151179]
Removed last element from the list:  [1, 0.99304865938431, 0.9706617486471405, 0.934056396477656, 0.8884870479156888, 0.8338851080567814, 0.7767730500853498, 0.7202296919196319, 0.666613464512311, 0.612385270513089]


In [35]:
# As list does not support element to element operations directly by using disc_fact_0Y_list_shift/disc_fact_0Y_list_orig, _
# both lists are converted to array to perform that operation
disc_fact_0Y_array_orig = np.array(disc_fact_0Y_list_orig)
disc_fact_0Y_array_shift = np.array(disc_fact_0Y_list_shift)

print(disc_fact_0Y_array_orig)
print(disc_fact_0Y_array_shift)

forward_rates_0Y_array = disc_fact_0Y_array_shift/disc_fact_0Y_array_orig-1
print(forward_rates_0Y_array)

[0.99304866 0.97066175 0.9340564  0.88848705 0.83388511 0.77677305
 0.72022969 0.66661346 0.61238527 0.55839478]
[1.         0.99304866 0.97066175 0.9340564  0.88848705 0.83388511
 0.77677305 0.72022969 0.66661346 0.61238527]
[0.007      0.02306356 0.03918966 0.0512887  0.06547897 0.07352477
 0.0785074  0.08043076 0.08855241 0.09668875]


For details on numpy arrays, refer to https://www.w3schools.com/python/numpy_intro.asp

###**3.** Calculation of single period forward rates at t = 1 and t = 2 based on forward rates at t = 0

**Future Scenario and calculation of expected spot curve from forward curve**

If the economy does not change, then forward rates as of today will be realized in the future.  

In such scenario, the forward rate curve in 1 year from now (containing n-1 term points) can be created by dropping the rate at index 0 in forward_rates_0Y_array. Therefore, forward_rates_1Y_array can be determined by applying forward_rates_0Y_array.pop(0) and would have n-1 term points. 

Similarly, forward_rates_2Y_array can be determined by applying forward_rates_1Y_array.pop(0) and would have n-2 term points, AND SO ON....

In [36]:
#Note that while an existing array can be assigned directly to a new variable, the lists need to be copied using list.copy(my_list) as shown above 
forward_rates_1Y_array = np.delete(forward_rates_0Y_array,0)
forward_rates_2Y_array = np.delete(forward_rates_1Y_array,0)
print("The expected forward rates in 1 year from now are: ",forward_rates_1Y_array)
print("The expected forward rates in 2 years from now are: ", forward_rates_2Y_array)

The expected forward rates in 1 year from now are:  [0.02306356 0.03918966 0.0512887  0.06547897 0.07352477 0.0785074
 0.08043076 0.08855241 0.09668875]
The expected forward rates in 2 years from now are:  [0.03918966 0.0512887  0.06547897 0.07352477 0.0785074  0.08043076
 0.08855241 0.09668875]


###**4.** Generation of single period forward discount factors at t = 0, t = 1, and t = 2

Note: Here also, the assumption is that the economy unfolds as it was expected as t = 0

In [37]:
single_period_disc_fact_0Y = 1/(1+ forward_rates_0Y_array)
single_period_disc_fact_1Y = 1/(1+ forward_rates_1Y_array)
single_period_disc_fact_2Y = 1/(1+ forward_rates_2Y_array)

print(single_period_disc_fact_0Y)
print(single_period_disc_fact_1Y)
print(single_period_disc_fact_2Y)

#The single period discount factors for t =1 and t = 2 can be created by popping out first element from t=0 one at a time.

[0.99304866 0.97745638 0.96228825 0.95121349 0.93854504 0.93151088
 0.92720736 0.92555677 0.91865122 0.91183574]
[0.97745638 0.96228825 0.95121349 0.93854504 0.93151088 0.92720736
 0.92555677 0.91865122 0.91183574]
[0.96228825 0.95121349 0.93854504 0.93151088 0.92720736 0.92555677
 0.91865122 0.91183574]


###**5.** Generation of spot discount factors at t = 1, and t = 2

In [38]:
#Creating a function to calculate spot curve at a forward date based on single period discount factors
def disc_fact_spot(single_period_disc_fact, forward_period_from_today):
  disc_fact = []
  cnt = 0
  spot_disc_factor = 1

  for i in range(nb_of_term_points-forward_period_from_today):
    spot_disc_factor = spot_disc_factor * single_period_disc_fact[i]
    disc_fact.append(spot_disc_factor)
  return disc_fact

In [39]:
disc_fact_spot_0Y = disc_fact_spot(single_period_disc_fact_0Y,0)
disc_fact_spot_0Y

[0.99304865938431,
 0.9706617486471404,
 0.9340563964776558,
 0.8884870479156886,
 0.8338851080567812,
 0.7767730500853496,
 0.7202296919196318,
 0.6666134645123108,
 0.612385270513089,
 0.5583947769151179]

In [40]:
disc_fact_spot_1Y = disc_fact_spot(single_period_disc_fact_1Y,1)
disc_fact_spot_1Y

[0.9774563808876703,
 0.9405947912529993,
 0.8947064572510982,
 0.8397223038131785,
 0.7822104614359469,
 0.7252712997630691,
 0.671279758763897,
 0.6166719674066806,
 0.5623035403535237]

In [41]:
disc_fact_spot_2Y = disc_fact_spot(single_period_disc_fact_2Y,2)
disc_fact_spot_2Y

[0.9622882510611928,
 0.91534156893894,
 0.8590892854477972,
 0.8002510155241791,
 0.7419986343579026,
 0.6867618564771953,
 0.630894615314347,
 0.5752722590473772]

###**6.**Generation of expected Spot rate curve at t = 0, t = 1, and t = 2 based on expected discount factor curve. 

Note: There are short cut methods to do this, but it is explained in detail to make things simple and logically connected

In [42]:
# 1 is added to start the tenors from 1 and not 0
def spot_disc_fact_to_spot_rate(nb_of_term_points, disc_fact_spot, forward_period_from_today):
  tenors = np.array(range(nb_of_term_points- forward_period_from_today))+1
  disc_fact_spot = np.array(disc_fact_spot)
  spot_rate_curve = (1/disc_fact_spot)**(1/tenors)-1
  return spot_rate_curve

In [43]:
spot_rate_curve_0Y = spot_disc_fact_to_spot_rate(nb_of_term_points, disc_fact_spot_0Y, 0)
spot_rate_curve_1Y = spot_disc_fact_to_spot_rate(nb_of_term_points, disc_fact_spot_1Y, 1)
spot_rate_curve_2Y = spot_disc_fact_to_spot_rate(nb_of_term_points, disc_fact_spot_2Y, 2)

print("Spot Curve Today: ", spot_rate_curve_0Y)
print("\nExpected Spot Curve after 1Y: ",spot_rate_curve_1Y)
print("\nExpected Spot Curve after 2Y: ",spot_rate_curve_2Y)

Spot Curve Today:  [0.007 0.015 0.023 0.03  0.037 0.043 0.048 0.052 0.056 0.06 ]

Expected Spot Curve after 1Y:  [0.02306356 0.03109508 0.03778282 0.04463862 0.05035299 0.05499383
 0.05859067 0.06229032 0.06605846]

Expected Spot Curve after 2Y:  [0.03918966 0.04522167 0.05193095 0.05728834 0.06149849 0.06463067
 0.0680156  0.07155834]


##**Forward time Price dynamics of bond in 1 year interval**

Here we analyze the impact of price assuming the interest rate rates are realized as per the expectations.

**For example: **

(1) 1f2 in year 0 becomes 0f1 (or spot rate 0S1) in year 1

(2) 2f3 in year 0 becomes 1f2 in year 1

(3) 2f3 in year 0 becomes 0f1 (or spot rate 0S1) in year 2, etc.

This is to show that typically in a duration neutral zero value portfolio, the leg with higher convexity will usually underperform the leg with lower convexity (for this reason, choosing the higher convexity by the trader may not result in overall profit in due course of time, especially, if the market does not change much)

Note: Here, the holding period for performance analysis is considered to be only 1 year, just to avoid the complexities of having a coupon payment that needs to be reinvested in the holding period and incorporated in the analysis. Incorporating that is easy though.

###**1.**Calculation of Individual bond prices at t = 0 and t = 1

Note: While the prices are already calculated at t = 0, this is done just for demonstration purposes with t = 1

In [44]:
bond_price_t0_5Y = bond_price_spot(cpn_5Y, spot_rate_curve_0Y, 5, prin_5Y)
bond_price_t1_5Y = bond_price_spot(cpn_5Y, spot_rate_curve_1Y, 4, prin_5Y)

print("5Y bond:")
print(bond_price_t0_5Y)
print(bond_price_t1_5Y)

bond_price_t0_2Y = bond_price_spot(cpn_2Y, spot_rate_curve_0Y, 2, prin_2Y)
bond_price_t1_2Y = bond_price_spot(cpn_2Y, spot_rate_curve_1Y, 1, prin_2Y)

print("\n2Y bond:")
print(bond_price_t0_2Y)
print(bond_price_t1_2Y)

bond_price_t0_8Y = bond_price_spot(cpn_8Y, spot_rate_curve_0Y, 8, prin_8Y)
bond_price_t1_8Y = bond_price_spot(cpn_8Y, spot_rate_curve_1Y, 7, prin_8Y)

print("\n8Y bond:")
print(bond_price_t0_8Y)
print(bond_price_t1_8Y)

5Y bond:
129.58990041049393
120.49702971336728

2Y bond:
110.8121477209342
104.58783275498072

8Y bond:
161.63391878921527
148.7653562207397


###**2.**Calculation of change in portfolio value due to time lapse between t = 0 and t = 1, assuming the rates are realized based on the forward expectations at t = 0

Note: Typically, the net portfolio change will be negative here. Thus, while the trader generates profit by going long on higher convexity and short on lower convexity based on the expectation of high volatility in interest rates (and thus high interest rate moves), if that expectation does not hold true, the loss from the passage of time might be higher than the profits generated by increased convexity.

Also, the assumption of parallel rate moves typically does not hold true. For quantifying PnLs based on parallel rate moves, the key rate duration and rope bucket techniques are used.

In [45]:
Price_chg_short = bond_price_t1_5Y - bond_price_t0_5Y
Price_chg_long =  n1* (bond_price_t1_2Y - bond_price_t0_2Y) + n2* (bond_price_t1_8Y - bond_price_t0_8Y)

Portfolio_Value_Chg = Price_chg_long - Price_chg_short
print("Change in short leg value: ","%.2f"% Price_chg_short)
print("Change in long leg value: ","%.2f"% Price_chg_long)
print("Change in Portfolio value: ","%.2f"% Portfolio_Value_Chg)

Change in short leg value:  -9.09
Change in long leg value:  -9.10
Change in Portfolio value:  -0.01


##**Calculation of forward YTM at t = 1 using Newton Raphson**

In [46]:
bond_ytm_5Y_t1 = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_5Y, 4, prin_5Y, 0.001, bond_price_t1_5Y)
bond_ytm_2Y_t1 = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_2Y, 1, prin_2Y, 0.001, bond_price_t1_2Y)
bond_ytm_8Y_t1 = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_8Y, 7, prin_8Y, 0.001, bond_price_t1_8Y)

print("The forward YTM at t = 1 for 5Y bond is: ","%.5f"% bond_ytm_5Y_t1)
print("The forward YTM at t = 1 for 2Y bond is: ","%.5f"% bond_ytm_2Y_t1)
print("The forward YTM at t = 1 for 8Y bond is: ","%.5f"% bond_ytm_8Y_t1)

The forward YTM at t = 1 for 5Y bond is:  0.04312
The forward YTM at t = 1 for 2Y bond is:  0.02306
The forward YTM at t = 1 for 8Y bond is:  0.05438
